In [ ]:
# This code shows the implementation and evaluation of a multivariate classifier for hearing loss detection 

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,roc_curve,auc,accuracy_score,precision_recall_curve

# import dataset and display first rows
df = pd.read_csv("FILE_PATH")
df.head()
# Descriptive statistics for each column
features.describe()

In [ ]:
# Save features matrix and labels vector
X = df.drop(['true_class'], axis = 1).values
y = df['true_class'].values

In [ ]:
# Function that returns the threshold corresponding to the closest point to (1,0) on the ROC curve 
def find_best_th_roc(ytrain, temp_ytrain_hat):
    fpr_train, tpr_train, th_roc_train = roc_curve(ytrain, temp_ytrain_hat)
    best_point_roc_x = np.array([0] * fpr_train.shape[0])
    best_point_roc_y = np.array([1] * tpr_train.shape[0])
    temp_x = (fpr_train - best_point_roc_x)
    temp_y = (tpr_train - best_point_roc_y)
    temp_sqrt = np.sqrt(np.square(temp_x) + np.square(temp_y))
    index_min_temp_sqrt = np.argmin(temp_sqrt)
    best_th_roc = th_roc_train[index_min_temp_sqrt]
    return best_th_roc

In [ ]:
n_iterations = 50
n_features = 6
u = np.zeros(n_features)
s = np.zeros(n_features)
u_vet = []
s_vet = []

AUC_tree_vet = []
acc_test_tree_vet = []
acc_train_tree_vet = []
spec_tree_vet=[]
sen_tree_vet=[]
precision_tree_vet=[]

AUC_forest_vet = []
acc_test_forest_vet = []
acc_train_forest_vet = []
spec_forest_vet=[]
sen_forest_vet=[]
precision_forest_vet=[]

# initialize DT and RF with previously tuned hyperparameters 
tree = DecisionTreeClassifier(criterion = 'gini',max_depth=4,random_state = 9)
forest = RandomForestClassifier(n_estimators = 50, criterion = 'gini', max_depth = None, min_samples_split = 2, min_samples_leaf = 1, min_weight_fraction_leaf = 0.0, max_features='auto', max_leaf_nodes = None, min_impurity_decrease = 0.0, bootstrap = True, oob_score = False, n_jobs = None, random_state = None, verbose = 0, warm_start = False, class_weight = None, ccp_alpha = 0.0, max_samples = None)
scaler = StandardScaler()
for i in range(0,n_iterations): # 50 iterations
    # Split the data into training (80%) and test (20%) sets
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, shuffle = True,  stratify = y) 
    
    '''
    #SAVE X train
    filename_Xtrain="FOREST/X_train/X_train_it"+str(i)+'.txt'
    Xtrain_df=pd.DataFrame(X_train)
    Xtrain_df.to_csv(filename_Xtrain,sep=',',header=False,index=False)
    
    #SAVE X test
    filename_Xtest="FOREST/X_test/X_test_it"+str(i)+'.txt'
    Xtest_df=pd.DataFrame(X_test)
    Xtest_df.to_csv(filename_Xtest,sep=',',header=False,index=False)
    
    #SAVE y train
    filename_ytrain="FOREST/y_train/y_train_it"+str(i)+'.txt'
    ytrain_df=pd.DataFrame(y_train)
    ytrain_df.to_csv(filename_ytrain,sep=',',header=False,index=False)
    
    #SAVE y test
    filename_ytest="FOREST/y_test/y_test_it"+str(i)+'.txt'
    ytest_df=pd.DataFrame(y_test)
    ytest_df.to_csv(filename_ytest,sep=',',header=False,index=False)
    '''
   
    # save mean and std to be able to rescale the data outside this notebook
    for ind in range (0, n_features):
        u[ind] = X_train[:,ind].mean()# u is the mean of the training samples
        s[ind] = X_train[:,ind].std()# s is the std of the training samples
    u_vet.append(u) 
    s_vet.append(s)  
    u = np.zeros(n_features)
    s = np.zeros(n_features)
    
    # Standard scaler
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    acc_train_best_forest=0
    acc_train_best_forest=0
    # nested k fold cross-validation
    kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 10) 

    for k, (train, valid) in enumerate (kfold.split(X_train, y_train)):
        #DT
        tree.fit(X_train[train], y_train[train])  # fit the model on training data
        score_tree_kfold_new = tree.score(X_train[valid], y_train[valid])
        if score_tree_kfold_new > acc_train_best_tree: # search for the best model 
            best_tree = tree.fit(X_train[train], y_train[train]) 
            acc_train_best_tree=score_tree_kfold_new
             
        #RF
        forest.fit(X_train[train], y_train[train])
        score_forest_kfold_new = forest.score(X_train[valid], y_train[valid])
        if score_forest_kfold_new > acc_train_best_forest:
            best_forest = forest.fit(X_train[train], y_train[train])
            acc_train_best_forest=score_forest_kfold_new
        
        
    '''Best DT model at iterations i'''
    # Save i-th model to a pickle file in the current working directory
    pkl_filename_tree = "tree/MODELS"/tree_it"+str(i)+".pkl"
    with open(pkl_filename_tree, 'wb') as file:
        pickle.dump(best_tree, file)  
    
    # Save DT in dot format for later visualization                          
    dotfile = open("DT_figures\DT"+str(i)+".dot", "w")
    export_graphviz(best_tree, out_file = dotfile, feature_names = ['SRT', 'age','#correct', '%correct','avg_reaction_time','total_test_time'],filled=True)
    dotfile.close()
    
    #evaluate classification performance
    acc_train_tree_vet.append(acc_train_best_tree)
    
    ytest_hat = best_tree.predict_proba(X_test)
    y_train_hat = best_tree.predict_proba(X_train)
    temp_ytrain_hat = np.copy(y_train_hat[:,1])
    best_th_roc = find_best_th_roc(y_train, temp_ytrain_hat)
    ytest_hat_l = (ytest_hat[:,1] >= best_th_roc).astype(int)
    
    #CONFUSION MATRIX on the test set
    (tn_tree, fp_tree, fn_tree, tp_tree) = confusion_matrix(y_test, ytest_hat_l).ravel()  
    accuracy_test = (tn_tree + tp_tree) / (tn_tree + tp_tree + fn_tree + fp_tree)
    acc_test_tree_vet.append(accuracy_test)    
    # Compute recall (sensitivity) = TP/(TP+FN) 
    sen_tree_vet.append(tp_tree / (tp_tree + fn_tree))    
    # Compute true negative rate (specificity) = TN /(TN+FP)
    spec_tree_vet.append(tn_tree / (tn_tree + fp_tree))    
    #Compute precision (positive predicted value) = TP/(TP+FP)
    precision_tree_vet.append(tp_tree  / (tp_tree  + fp_tree )    
    #Compute AUC
    fpr, tpr, th_roc = roc_curve(y_test, ytest_hat[:,1])
    roc_auc = auc(fpr, tpr)
    AUC_tree_vet.append(roc_auc)
    
                              
    '''Best RF model at iterations i'''
    # Save i-th model to a pickle file in the current working directory
    pkl_filename_Forest = "FOREST/MODELS"/Forest_it"+str(i)+".pkl"
    with open(pkl_filename_Forest, 'wb') as file:
        pickle.dump(best_forest, file)   
        
    #evaluate classification performance
    acc_train_forest_vet.append(acc_train_best_forest)
    
    ytest_hat = best_forest.predict_proba(X_test)
    y_train_hat = best_forest.predict_proba(X_train)
    temp_ytrain_hat = np.copy(y_train_hat[:,1])
    best_th_roc = find_best_th_roc(y_train, temp_ytrain_hat)
    ytest_hat_l = (ytest_hat[:,1] >= best_th_roc).astype(int)
    
    #CONFUSION MATRIX on the test set
    (tn_forest, fp_forest, fn_forest, tp_forest) = confusion_matrix(y_test, ytest_hat_l).ravel()  
    accuracy_test = (tn_forest + tp_forest) / (tn_forest + tp_forest + fn_forest + fp_forest)
    acc_test_forest_vet.append(accuracy_test)    
    # Compute recall (sensitivity) = TP/(TP+FN) 
    sen_forest_vet.append(tp_forest / (tp_forest + fn_forest))    
    # Compute true negative rate (specificity) = TN /(TN+FP)
    spec_forest_vet.append(tn_forest / (tn_forest + fp_forest))    
    #Compute precision (positive predicted value) = TP/(TP+FP)
    precision_forest_vet.append(tp_forest  / (tp_forest  + fp_forest )    
    #Compute AUC
    fpr, tpr, th_roc = roc_curve(y_test, ytest_hat[:,1])
    roc_auc = auc(fpr, tpr)
    AUC_forest_vet.append(roc_auc)
    
#save features mean and std (for each trainining set) for inverse normalization
df_mean = pd.DataFrame(u_vet,columns=['SRT_mean','Age_mean','correct_mean','percentage_mean','Avg_reaction_time_mean','Total_test_time_mean'])
df_std= pd.DataFrame(s_vet,columns=['SRT_std','Age_std','correct_std','percentage_std','Avg_reaction_time_std','Total_test_time_std'])
df_mean.to_csv("FOREST/mean_normalization.txt",sep=',',header=True,index=False)
df_std.to_csv("FOREST/std_normalization.txt",sep=',',header=True,index=False)'''


In [ ]:
'''Average DT classification performance over 50 iterations'''
print ('DT classification performance:')
print('Train accuracy: '+str(round(np.array(acc_train_tree_vet).mean(),3))+' ±  '+str(round(np.array(acc_train_tree_vet).std(),2)))
print('Test accuracy: '+str(round(np.array(acc_test_tree_vet).mean(),3))+' ±  '+str(round( np.array(acc_test_tree_vet).std(),2)))
print('AUC: '+str(round(np.array(AUC_tree_vet).mean(),3))+' ±  '+str(round(np.array(AUC_tree_vet).std(),2)))
print('Specificity: '+str(round(np.array(spec_tree_vet).mean(),3))+' ±  '+str(round(np.array(spec_tree_vet).std(),2)))
print('Sensitivity: '+str(round(np.array(sen_tree_vet).mean(),3))+' ±  '+str(round(np.array(sen_tree_vet).std(),2)))
print('Precision: '+str(round(np.array(precision_tree_vet).mean(),3))+' ±  '+str(round(np.array(precision_tree_vet).std(),2)))

'''Average RF classification performance over 50 iterations'''
print ('RF classification performance:')
print('Train accuracy: '+str(round(np.array(acc_train_forest_vet).mean(),3))+' ±  '+str(round(np.array(acc_train_forest_vet).std(),2)))
print('Test accuracy: '+str(round(np.array(acc_test_forest_vet).mean(),3))+' ±  '+str(round( np.array(acc_test_forest_vet).std(),2)))
print('AUC: '+str(round(np.array(AUC_forest_vet).mean(),3))+' ±  '+str(round(np.array(AUC_forest_vet).std(),2)))
print('Specificity: '+str(round(np.array(spec_forest_vet).mean(),3))+' ±  '+str(round(np.array(spec_forest_vet).std(),2)))
print('Sensitivity: '+str(round(np.array(sen_forest_vet).mean(),3))+' ±  '+str(round(np.array(sen_forest_vet).std(),2)))
print('Precision: '+str(round(np.array(precision_forest_vet).mean(),3))+' ±  '+str(round(np.array(precision_forest_vet).std(),2)))